# Flows distribution on a road network

Install libadalina-analytics using `pip`

In [1]:
pip install libadalina-analytics

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Sedona context is initialized automatically when functions from libadalina are called. However, you can explicitly call the initialization procedure to customize options for the Spark session, such as memory allocation, parallelism settings, and other Spark configurations.


In [2]:
from libadalina_core.sedona_configuration import init_sedona_context

init_sedona_context()

https://artifacts.unidata.ucar.edu/repository/unidata-all added as a remote repository with the name: repo-1
Ivy Default Cache set to: /home/jupyter-unimi.uno/.ivy2/cache
The jars for the packages stored in: /home/jupyter-unimi.uno/.ivy2/jars
org.apache.sedona#sedona-spark-3.3_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ebc37704-43d7-432c-ba07-0cce5611a69e;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/jupyter-unimi.uno/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.sedona#sedona-spark-3.3_2.12;1.7.1 in central
	found org.apache.sedona#sedona-common;1.7.1 in central
	found org.apache.commons#commons-math3;3.6.1 in central
	found org.locationtech.jts#jts-core;1.20.0 in central
	found org.wololo#jts2geojson;0.16.1 in central
	found org.locationtech.spatial4j#spatial4j;0.8 in central
	found com.google.geometry#s2-geometry;2.0.0 in central
	found com.google.guava#guava;25.1-jre in central
	found com.google.code.findbugs#jsr305;3.0.2 in central
	found org.checkerframework#checker-qual;2.0.0 in central
	found com.google.errorprone#error_prone_annotations;2.1.3 in central
	found com.google.j2objc#j2objc-annotations;1.1 in central
	found org.codehaus.mojo#animal-sniffer-annotations;1.14 in central
	found com.uber#h3;4.1.1 in central
	found net.sf.geographiclib#GeographicLib-Java;1.52 in central
	found com.github.ben-manes.caffeine#caffeine;2.9.2 in central
	found org.checkerframework#checker-qual;3.10.0 in central
	found com.google.error

25/08/31 12:59:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Import the Amelia toolkit to read datasets.

In [3]:
from ameliadp_sql_toolkit import GrinsAmeliaSQLToolkit 
amelia_sql_toolkit = GrinsAmeliaSQLToolkit()

Read and parse the OpenStreetMap geometries dataset containing the roadmap network of Milano. This dataset includes road segments with their geographical coordinates and properties.

The output of the `OpenStreetMapReader` is a new dataset with only the columns that are necessary for the graph building process.

In [4]:
from libadalina_core.graph_extraction.readers import OpenStreetMapReader, RoadTypes

milano_roadmap_df = amelia_sql_toolkit.read_data(table_name='milano_roadmap')
osm_df = OpenStreetMapReader(RoadTypes.CAR_ONLY).from_dataframe(milano_roadmap_df)
osm_df.head()

Digit your password here:  ········


/home/jupyter-unimi.uno/.local/lib/python3.10/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,geometry,id,name,oneway
0,"LINESTRING (9.16836 45.47604, 9.16827 45.47595...",4011790,Via Antonio Canova,forward
1,"LINESTRING (9.15073 45.45995, 9.15066 45.45993...",4011792,Via Costanza,forward
2,"LINESTRING (9.1711 45.47085, 9.17106 45.47089,...",4011793,Viale Pietro e Maria Curie,forward
3,"LINESTRING (9.15084 45.46181, 9.15076 45.46187...",4011799,Via Marchesi de' Taddei,forward
4,"LINESTRING (9.14642 45.47797, 9.14631 45.47805...",4011800,Via Monte Bianco,forward


Read the dataset containing population data. This dataset is structured as a spatial grid, with each cell containing population density information for its corresponding geographical area.


In [5]:
population = amelia_sql_toolkit.read_data(table_name='milano_population')[['t', 'geometry']]
population.head()

,t,geometry
0,85,"POLYGON ((9.031086950078857 45.38280269166012,..."
1,10,"POLYGON ((9.04383459344471 45.38291898197039, ..."
2,0,"POLYGON ((9.056582284984122 45.38303373224426,..."
3,4,"POLYGON ((9.06933002405527 45.38314694247239, ..."
4,46,"POLYGON ((9.082077810016315 45.38325861264565,..."


Convert both OpenStreetMap and population datasets to Spark DataFrame to set the EPSG code of the geometries.
If not set explicitly, libadalina will try to infer the code from the geometries.

In [6]:
from libadalina_core.sedona_utils import to_spark_dataframe, EPSGFormats

osm_df = to_spark_dataframe(osm_df, EPSGFormats.EPSG4326)
population = to_spark_dataframe(population, EPSGFormats.EPSG4326)

Run the graph building process:
First, the roadmap dataset is first joined with the population grid in such a way that each edge is enriched with the amount of population living within a distance of 1km.
Then lines of the map are converted into edges of a `networkx` `DiGraph`.

Also, each edge is automatically enriched with its length in meters.

In [7]:
from libadalina_core.graph_extraction.builders import build_graph
from libadalina_core.spatial_operators import AggregationFunction, AggregationType

graph = build_graph(osm_df,
                    name='milan_road',
                    joined_df=population,
                    buffer_radius_meters=1000, # 1km buffer around the roads to consider population
                    aggregate_functions=[
                        AggregationFunction("t", AggregationType.SUM, 'population', proportional='geometry_right')
                    ]
                    )

25/08/31 13:00:04 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


In [8]:
graph.number_of_nodes(), graph.number_of_edges()

(146760, 220279)

We read the dataset containing the flows of goods within different regions of Lombardia and aggregate columns in a single demand.

In [9]:
flows = amelia_sql_toolkit.read_data(table_name='matrice_od_2016_merci')
flows['demand'] = flows.apply(lambda row: row['n1'] + row['n2'] + row['n3'], axis=1)
flows.head()

,z_iil_o,z_iil_o_nome,z_iil_d,z_iil_d_nome,n1,n2,n3,metadata_vd,metadata_vtd,metadata_ud,demand
0,1494,SICILIA,98,LECCO,0.00,0.08,0.07,1,2025-08-31 11:08:47.82082,2025-08-31,0.15
1,1495,SARDEGNA,1469,ROVIGO,0.00,0.13,0.09,1,2025-08-31 11:08:47.82082,2025-08-31,0.22
2,1,ARCENE-LURANO-POGNANO-SPIRANO,10,BERGAMO 1-BERGAMO 7,1.00,0.09,0.12,1,2025-08-31 11:08:47.82082,2025-08-31,1.21
3,1,ARCENE-LURANO-POGNANO-SPIRANO,11,BERGAMO 5-BERGAMO 6,1.13,0.09,0.13,1,2025-08-31 11:08:47.82082,2025-08-31,1.35
4,1,ARCENE-LURANO-POGNANO-SPIRANO,12,BERGAMO 2-BERGAMO 3-BERGAMO 4,0.68,0.09,0.05,1,2025-08-31 11:08:47.82082,2025-08-31,0.82


We read the shapes of the different regions of Lombardia.
Each regions has a unique identifier also used in the flows' dataset.

In [10]:
shapes = amelia_sql_toolkit.read_data(table_name='shape_matrice_od2016___veicoli_commerciali_e_pesanti___milano_20250816')
shapes.head()

,id_z_iil,desc_zona,geometry,metadata_vd,metadata_vtd,metadata_ud
0,215,MILANO 1-MILANO 2-MILANO 3-MILANO 4-MILANO 7-M...,MULTIPOLYGON (((9.171230961752443 45.500721356...,1,2025-08-31 10:09:35.769394,2025-08-31
1,218,MILANO 15,MULTIPOLYGON (((9.157156769311907 45.493743652...,1,2025-08-31 10:09:35.769394,2025-08-31
2,219,MILANO 5-MILANO 16,MULTIPOLYGON (((9.175653934000579 45.535973345...,1,2025-08-31 10:09:35.769394,2025-08-31
3,220,MILANO 6-MILANO 8,MULTIPOLYGON (((9.268427923604095 45.516767840...,1,2025-08-31 10:09:35.769394,2025-08-31
4,221,MILANO 9-MILANO 10,MULTIPOLYGON (((9.234996403412737 45.447069452...,1,2025-08-31 10:09:35.769394,2025-08-31


Since the shape dataframe include geometries, we set the correct EPSG code.

In [11]:
shapes = to_spark_dataframe(shapes, EPSGFormats.EPSG4326)

We run the flow distribution algorithm to find optimal paths for goods transportation between source-destination pairs from the flows dataset.

In this example, we calculates costs for each potential path by considering:
- Distance traveled (30% weight): shorter paths are preferred
- Population exposure (70% weight): routes that minimize the number of people living within 1km of the path are preferred



In [12]:
from libadalina_analytics.flows_distribution.algorithms.flows_distribution_algorithm import flows_distribution_algorithm, GraphCost

result = flows_distribution_algorithm(graph, shapes, flows,
                             graph_costs=[
                                 GraphCost(name='population', cost_per_unit=1, weight=.7),
                                 GraphCost(name='distance', cost_per_unit=1, weight=.3),
                             ],
                             shapes_id_column='id_z_iil',
                             flows_origin_id_column='z_iil_o',
                             flows_destination_id_column='z_iil_d',
                             flows_demand_column='demand',
                         )

25/08/31 13:00:54 WARN RangeJoinExec: [SedonaSQL] Join dominant side partition number 24 is larger than 1/2 of the dominant side count 6
25/08/31 13:00:54 WARN RangeJoinExec: [SedonaSQL] Try to use follower side partition number 24
25/08/31 13:00:54 WARN RangeJoinExec: [SedonaSQL] Join follower side partition number is also larger than 1/2 of the dominant side count 6
25/08/31 13:00:54 WARN RangeJoinExec: [SedonaSQL] Try to use 1/2 of the dominant side count 3 as the partition number of both sides
25/08/31 13:00:54 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


The result is the flows dataset where each row is enriched with
* the sequence of graph nodes traversed in the path
* the geometry of the path
* the cost of the path
* the distance traveled
* the amount of population exposed

In [13]:
result.head()

,z_iil_o,z_iil_d,z_iil_o_nome,z_iil_d_nome,n1,n2,n3,metadata_vd,metadata_vtd,metadata_ud,demand,path,geometry,path_cost,population,distance
0,220,215,MILANO 6-MILANO 8,MILANO 1-MILANO 2-MILANO 3-MILANO 4-MILANO 7-M...,131.21,18.32,12.97,1,2025-08-31 11:08:47.82082,2025-08-31,162.50,"[146028888666, 25769804389, 111669150243, 2061...","MULTILINESTRING ((9.24654 45.47253, 9.2465 45....",1.591292e+07,137345.628524,5945.824494
1,221,215,MILANO 9-MILANO 10,MILANO 1-MILANO 2-MILANO 3-MILANO 4-MILANO 7-M...,60.17,7.53,4.76,1,2025-08-31 11:08:47.82082,2025-08-31,72.46,"[120259089516, 111669155190, 103079220324, 858...","MULTILINESTRING ((9.20464 45.42008, 9.20454 45...",5.839698e+06,112283.591699,6645.021379
2,222,221,MILANO 11-MILANO 13,MILANO 9-MILANO 10,28.69,3.95,3.42,1,2025-08-31 11:08:47.82082,2025-08-31,36.06,"[3468, 146028891512, 60129543658, 120259086168...","MULTILINESTRING ((9.11054 45.4554, 9.11033 45....",2.706023e+06,98945.421866,19268.126156
3,218,221,MILANO 15,MILANO 9-MILANO 10,15.71,2.01,1.41,1,2025-08-31 11:08:47.82082,2025-08-31,19.13,"[68719479552, 111669152135, 85899348400, 11166...","MULTILINESTRING ((9.12055 45.50178, 9.12057 45...",1.756212e+06,114213.876945,39514.553648
4,218,222,MILANO 15,MILANO 11-MILANO 13,19.91,2.10,2.09,1,2025-08-31 11:08:47.82082,2025-08-31,24.10,"[68719479552, 111669152135, 85899348400, 11166...","MULTILINESTRING ((9.12055 45.50178, 9.12057 45...",1.495737e+06,85107.332526,8295.506683
